# Caltech 40 m Interferometer

In [ ]:
# Import and initialize matlab engine.
# This takes some time to initialize matlab, so only run it once at startup.
import matlab.engine
eng = matlab.engine.start_matlab()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

In [ ]:
mpl.rc('figure', figsize=(12, 9))
mpl.rcParams.update({'text.usetex': False,
                     'font.family': 'serif',
                     'font.serif': 'Georgia',
                     'mathtext.fontset': 'cm',
                     'lines.linewidth': 2,
                     'font.size': 16,
                     'legend.loc': 'best',
                     'savefig.dpi': 80,
                     'pdf.compression': 9})

In [ ]:
import qlance.optickle as pyt
import opt40m
import param40m

In [ ]:
# Add the optickle path
pyt.addOpticklePath(eng)

## Define and Run the Model

In [ ]:
ff = np.logspace(1, 3.5, 200)  # frequency vector [Hz]

DARM = {'EX': 1, 'EY': -1}

In [ ]:
# Define parameters
Larm = 40  # arm length [m]
Pin = 10  # input power [W]
Tsrm = 0.25  # SRM transmisivity
phi = 0.2  # SRC detuning [deg]
zeta = 88  # homodyne angle [deg]
par = param40m.param40m(Larm=Larm, Tsrm=Tsrm)

In [ ]:
# Define the model
opt = opt40m.opt40m(eng, 'opt', par, phi, zeta, Pin)

In [ ]:
# Compute the model
opt.run(ff)

## Make automatic plots

In [ ]:
# Plot the DARM transfer function
fig = opt.plotTF('AS_DIFF', DARM);
fig.axes[0].set_ylabel('Magnitude [W/m]');
fig.axes[0].set_title('DARM Response');
fig.set_size_inches(10, 12);

In [ ]:
# Plot the quantum noise
fig = opt.plotQuantumASD('AS_DIFF', DARM);
fig.gca().set_title('Quantum Noise');

## Get data and make custom plots

In [ ]:
tf = opt.getTF('AS_DIFF', DARM)  # DARM transfer function [W/m]
qnoise = opt.getQuantumNoise('AS_DIFF')  # quantum noise [W/sqrtHz]
Squant = qnoise / np.abs(tf)  # quantum noise [m/rtHz]
Squant /= Larm

In [ ]:
fig = plt.figure();
ax = fig.gca();
ax.loglog(ff, Squant);
ax.set_xlim(ff[0], ff[-1]);
ax.set_xlabel('Frequency [Hz]');
ax.set_ylabel(r'Strain $[1/\mathrm{Hz}^{1/2}]$');
ax.set_title('Quantum Noise');
ax.grid(True, which='both', alpha=0.5);
ax.grid(which='minor', alpha=0.25);